In [13]:
from pathlib import Path
import pandas as pd
import json
from pandas.api.types import union_categoricals
from itertools import islice
import re

In [14]:
oews_path = Path('../Data/oews')

In [15]:
# Create dict with list of MSA files within each folder-year
oews_path_dict = {}
for folder in Path(oews_path).iterdir():
    folder_name = folder.name
    year = re.findall(r'\d+', folder_name)[0]
    
    if year == '97' or year == '98' or year == '99':
        year = '19' + year
    else:
        year = '20' + year

    file_list = []
    for file in Path(folder).iterdir():
        file_name = file.name
        file_prefix = file_name[0:3]

        # All MSA files is prefixed with 'msa' or 'MSA', except for 1997, which has 'oes' prefix
        # Non-metro areas are in separate file with 'BOS' prefix beginning in 2006
        if file_prefix == 'msa' or file_prefix == 'MSA' or file_prefix == 'oes' or file_prefix == 'BOS':
            file_list.append(file)

    oews_path_dict[year] = file_list

In [16]:
# Directory we are storing processed dataframes in binary format, for quick access later
binary_path = Path('../binaries/')
binary_path.mkdir(parents = True, exist_ok = True)

# Dict to store paths to processed binaries
oews_binary_path_dict = {}

In [17]:
# These variables we want to store as strings
oews_dtype_dict = {
    'area': 'string',
    'area_name': 'string',
    'occ_code': 'string',
    'occ_title': 'string'
}

In [18]:
oews_df = pd.DataFrame()
for year, file_list in oews_path_dict.items():
    file_list_df_dict = {} # Dict to store all relevant dfs in each year-folder to be concatenated into a single year df

    print(f'Loading files for year {year}.')
    
    for file_path in file_list:

        # Prior to 2001, field descriptions were placed in the header rows

        if int(year) < 2001:
            df = pd.read_excel(file_path, dtype = str)

            header_row = df['Unnamed: 0'].isna().values.argmin() # Find first row that is non-empty in first column, which should be the header row
            header = df.iloc[header_row]

            # Create new df without field description and header rows, and new headers
            new_df = df.iloc[header_row + 1:]
            new_df = new_df.rename(columns = header)
            
        else:
            new_df = pd.read_excel(file_path, header = 0, dtype = str)

        # Keep only columns we want and store in df dict
        # For year 2000, occ_title has a typo
        if int(year) == 2000:
            new_df.rename(columns = {'occ_titl': 'occ_title'}, inplace = True)

        new_df.columns = new_df.columns.str.lower() # Convert to lowercase column titles for all years

        # For years 1998-2002, wage percentile columns had 'h_wpct' and 'a_wpct' prefixes
        if 1997 < int(year) < 2003:
            new_df.rename(columns = {'h_wpct10': 'h_pct10', 'h_wpct25': 'h_pct25', 'h_wpct75': 'h_pct75', 'h_wpct90': 'h_pct90', 'a_wpct10': 'a_pct10', 'a_wpct25': 'a_pct25', 'a_wpct75': 'a_pct75', 'a_wpct90': 'a_pct90'}, inplace = True)

        # Starting in 2019, area_name was changed to area_title
        if int(year) >= 2019:
            new_df.rename(columns = {'area_title': 'area_name'}, inplace = True)

        if int(year) != 1997:
            new_df = new_df[['area', 'area_name', 'occ_code', 'occ_title', 'tot_emp', 'h_mean', 'a_mean', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median', 'a_pct75', 'a_pct90']]
        else:
            new_df = new_df[['area', 'area_name', 'occ_code', 'occ_title', 'tot_emp', 'h_mean', 'a_mean', 'h_median']]
        
        file_list_df_dict[file_path.name] = new_df

    combined_df = pd.concat([df for df in file_list_df_dict.values()]) # Concat all dfs into one
    combined_df.reset_index(drop = True, inplace = True)

    # Convert total employment counts and mean wages into numeric
    combined_df['tot_emp'] = pd.to_numeric(combined_df['tot_emp'], errors = 'coerce')
    combined_df['a_mean'] = pd.to_numeric(combined_df['a_mean'], errors = 'coerce')

    # Area code may have leading whitespace for 1999 and 2000
    combined_df['area'] = combined_df['area'].str.strip(' ')

    # Add year column
    combined_df['year'] = int(year)

    # Concat all years into one df
    oews_df = pd.concat([oews_df, combined_df], ignore_index=True)


Loading files for year 2000.
Loading files for year 2001.
Loading files for year 2002.
Loading files for year 1997.
Loading files for year 1998.
Loading files for year 1999.
Loading files for year 2003.
Loading files for year 2004.
Loading files for year 2005.
Loading files for year 2006.
Loading files for year 2007.
Loading files for year 2008.
Loading files for year 2009.
Loading files for year 2010.
Loading files for year 2011.
Loading files for year 2012.
Loading files for year 2013.
Loading files for year 2014.
Loading files for year 2015.
Loading files for year 2016.
Loading files for year 2017.
Loading files for year 2018.
Loading files for year 2019.
Loading files for year 2020.
Loading files for year 2021.


In [19]:
# Save combined df as parquet
target_path = binary_path.joinpath('oews.parquet')
oews_df.to_parquet(target_path)